Testing the code from Kjersti`s github calculating the mixed layer depth. 

In [14]:
#CODE implemented from Kjersti:
#from roppy import SGrid
from netCDF4 import Dataset
import numpy as np
from scipy.interpolate import griddata
from glob import glob
import time
import xarray as xr
import sys
import os
from dens_func import dens

In [7]:
def MLD(pot_dens, z):
    '''
    Calculate Mixed Layer Depth (MLD) based on potential density profile and depth.
    MLD is where the potential density exceeds a threshold, here set to be surface
    potential density + 0.03 kgm-3.

    Parameters:
    - pot_dens: 1D numpy array of potential density [kg/m^3]
    - z: 1D numpy array of corresponding depth levels [m] (negative downward)

    Returns:
    - mld: scalar value of MLD [m], or local water depth if no depth exceeds threshold,
    meaning full water column is mixed.
    '''
    # Remove NaNs
    valid = ~np.isnan(pot_dens)
    pot_dens = pot_dens[valid]
    z = z[valid]

    if len(pot_dens) == 0:
        return np.nan

    # Surface density
    surface_density = pot_dens[0]
    threshold = surface_density + 0.03  # MLD is where density exceeds surface + 0.03

    # Find where density exceeds threshold
    exceed = np.where(pot_dens >= threshold)[0]

    if exceed.size == 0:
        return z[-1]  # no depth exceeds threshold

    # Return the first depth where threshold is exceeded
    return z[exceed[0]]


In [9]:
ds_2024 = xr.open_dataset(f'/lustre/storeB/project/fou/hi/foccus/datasets/symlinks/norkystv3-hindcast/2024/norkyst800-20240330.nc')

In [12]:
ds_2024.head()

<xarray.Dataset> Size: 18kB
Dimensions:           (time: 5, Y: 5, X: 5, s_rho: 5, s_w: 5)
Coordinates:
  * time              (time) datetime64[ns] 40B 2024-03-30 ... 2024-03-30T04:...
  * s_rho             (s_rho) float64 40B -0.9799 -0.9403 ... -0.8634 -0.8261
  * X                 (X) float64 40B 0.0 800.0 1.6e+03 2.4e+03 3.2e+03
  * Y                 (Y) float64 40B 0.0 800.0 1.6e+03 2.4e+03 3.2e+03
  * s_w               (s_w) float64 40B -1.0 -0.96 -0.9208 -0.8823 -0.8446
    lon               (Y, X) float64 200B ...
    lat               (Y, X) float64 200B ...
Data variables: (12/18)
    Uwind_eastward    (time, Y, X) float32 500B ...
    Vwind_northward   (time, Y, X) float32 500B ...
    zeta              (time, Y, X) float32 500B ...
    ubar_eastward     (time, Y, X) float32 500B ...
    vbar_northward    (time, Y, X) float32 500B ...
    u_eastward        (time, s_rho, Y, X) float32 2kB ...
    ...                ...
    hc                float64 8B ...
    Cs_r              (s_rho) float64 40B ...
    Cs_w              (s_w) float64 40B ...
    h                 (Y, X) float32 100B ...
    projection_stere  int32 4B ...
    depth             int32 4B ...
Attributes: (12/33)
    id:                      15f95603-12d1-4e0f-8cbe-33946594447f
    naming_authority:        no.met
    operational_status:      scientific
    iso_topic_category:      oceans
    activity_type:           Numerical Simulation
    keywords_vocabulary:     GCMDSK:GCMD Science Keywords:https://gcmd.earthd...
    ...                      ...
    project:                 Norkyst_v3
    license:                 https://spdx.org/licenses/CC-BY-4.0 (CC-BY-4.0)
    title:                   Norkyst-800m - ROMS, Norkyst-800m ocean hindcast...
    summary:                 Norkyst-800m (Norwegian Coast 800m horizontal re...
    title_no:                Hindcast prognoser fra havmodellen Norkyst-800m,...
    summary_no:              NorKyst-800m (Norske kystområder med 800m horiso...